In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df=pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

In [ ]:
df.isna().sum()
totalmiss=pd.DataFrame(data=[df.columns,df.isna().sum()/df.shape[0]])
totalmiss


The data seems okay with no much missing features. 

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
oe=OrdinalEncoder()
si=SimpleImputer(strategy='most_frequent')
df=pd.DataFrame(si.fit_transform(df),columns=list(df))
df=pd.DataFrame(oe.fit_transform(df),columns=list(df))

In [ ]:
df.info()

The problems of categorical data and nan values are solved by now.

# Analysis

In [ ]:
df.corr()['BAD'].sort_values(ascending=False)

The correlation did not entertain much though. There is still a way!

 # Preparation

In [ ]:
x_train=df.drop(columns=['BAD'],axis=1)
y_train=df['BAD']

In [ ]:
q=pd.DataFrame(data=[y_train.value_counts(),y_train.value_counts()/y_train.shape[0]])
print(q)

The data is enormously skewed. I will be using SMOTE to balance it.

In [ ]:
from imblearn.over_sampling import SMOTE
smt=SMOTE()
x_train,y_train=smt.fit_resample(x_train,y_train)

Let's check the ratio now.

In [ ]:
q=pd.DataFrame(data=[y_train.value_counts(),y_train.value_counts()/y_train.shape[0]])
print(q)

The data seems balanced! Time for a little preprocessing!

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

## LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix
lr=LogisticRegression()
lr.fit(x_train,y_train)
y_pred=lr.predict(x_test)

print('Accuracy Score :',accuracy_score(y_test,y_pred))
print('F1_Score :',f1_score(y_test,y_pred))
print('ROC_AUC_Score',roc_auc_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))


0.74 is not a good accuracy score !!!

## XGBoost

In [ ]:
import xgboost
xgb=xgboost.XGBClassifier(max_depth=10,n_estimators=100)
xgb.fit(x_train,y_train)
y_pred=xgb.predict(x_test)


print('Accuracy Score :',accuracy_score(y_test,y_pred))
print('F1_Score :',f1_score(y_test,y_pred))
print('ROC_AUC_Score',roc_auc_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

Wow! The accuracy seemed to have skyrocketed! We can trust this figure, as the f1_score also seems pretty good!

Let's try LightGBM as a last method!

## LightGBM


In [ ]:
import lightgbm
lgbm=lightgbm.LGBMClassifier()
lgbm.fit(x_train,y_train)
y_pred=lgbm.predict(x_test)

print('Accuracy Score :',accuracy_score(y_test,y_pred))
print('F1_Score :',f1_score(y_test,y_pred))
print('ROC_AUC_Score',roc_auc_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

This gave a bit low score, but we can consider this amount of precision. 


## As a sidenote:
You can always try the problem without SMOTE and see the changes. In my experiment, the accuracy figures before applying SMOTE to the data were around 0.92 (LightGBM) with a very disappointing f1_score of 0.80 
